## **Merging images and annotations among teammates**


In [52]:
from pathlib import Path
from PIL import Image
import random, re, json, shutil
import xml.etree.ElementTree as ET

# defining directories
# ROOT = Path("dataset")
ROOT = Path("data_labelling")
MERGED = Path("merged_dataset")
IMG_EXTS = {".png"}

# config for balancing not_rubbish images with rubbish images
ADD_NOT_RUBBISH_BALANCED = True
# not_rubbish:rubbish ratio
MAX_NOT_RUBBISH_PER_POSITIVE = 0.1 # 1 here means 1:1 ratio

# train/val split ratio
TRAIN_RATIO = 0.8

# random seed for deterministic splits
random.seed(42)

# creating merged directories
(MERGED / "images" / "train").mkdir(parents=True, exist_ok=True)
(MERGED / "images" / "val").mkdir(parents=True, exist_ok=True)
(MERGED / "labels" / "train").mkdir(parents=True, exist_ok=True)
(MERGED / "labels" / "val").mkdir(parents=True, exist_ok=True)

print("setup success")

setup success


In [5]:
# labelme to YOLO conversion helpers
def unique_name(teammate_name: str, orig_name: str) -> str:
    """
    create a unique, filesystem-friendly base name: teammate_origbasename (no extension)
    prevents file overwriting
    """
    base = Path(orig_name).stem
    # using regex sanitize teammate name and base to avoid spaces/slashes
    t = re.sub(r'[^a-zA-Z0-9_-]+', '_', teammate_name.strip())
    b = re.sub(r'[^a-zA-Z0-9_-]+', '_', base)
    return f"{t}_{b}"


def bbox_from_points(points):
    """
    LabelMe: shape.points can be 2 points (rect) or a polygon list
    return(xmin, ymin, xmax, ymax)
    """

    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    return min(xs), min(ys), max(xs), max(ys)


def to_yolo_line(cls_id, bbox, img_w, img_h):
    """
    YOLO labels must be normalized to [0, -1] and in center-width-height format.
    """
    xmin, ymin, xmax, ymax = bbox

    # "clamp" coords to image bounds (no negatives/out-of-range)
    xmin, ymin = max(0, xmin), max(0, ymin)
    xmax, ymax = min(img_w, xmax), min(img_h, ymax)
    
    # convert to YOLO (normalized)
    xc = ((xmin + xmax) / 2) / img_w
    yc = ((ymin + ymax) / 2) / img_h
    w = (xmax - xmin) / img_w
    h = (ymax - ymin) / img_h

    # return in a format the YOLO expects
    return f"{cls_id} {xc} {yc} {w} {h}"


def is_image_file(p: Path) -> bool:
    return p.suffix.lower() in IMG_EXTS

In [6]:
# finding teammates folders, collect files
teammates = [d for d in ROOT.iterdir() if d.is_dir()]
print("teammate folders:", [t.name for t in teammates])

teammate folders: ['Ayush Bhandari', 'Keshini Munasinghe', 'Khanh Toan Nguyen', 'Max Harrison', 'Ratanakvisal Heng']


#### adding support for both JSON and XML annotations


In [7]:
# parse json annotations function
def parse_labelme_json(jp):
    """Parse LabelMe JSON -> unified dict."""
    data = json.loads(jp.read_text())
    # ensure required keys exist
    data.setdefault("imagePath", None)
    data.setdefault("imageWidth", data.get("imageWidth"))
    data.setdefault("imageHeight", data.get("imageHeight"))
    # LabelMe already has shapes with 'points'
    # we only keep rectangle/polygon by extracting 2-point rectangles for downstream
    unified_shapes = []
    for sh in data.get("shapes", []):
        lab = sh.get("label")
        pts = sh.get("points", [])
        if not lab or not pts:
            continue

        # normalize to 2-point rectangle [[xmin, ymin], [xmax, ymax]]
        xs = [p[0] for p in pts]
        ys = [p[1] for p in pts]
        rect = [[float(min(xs)), float(min(ys))], [float(max(xs)), float(max(ys))]]
        unified_shapes.append({"label": lab, "points": rect})
    
    return {
        "imagePath": data.get("imagePath"),
        "imageWidth": data.get("imageWidth"),
        "imageHeight": data.get("imageHeight"),
        "shapes": unified_shapes,
    }


# parse XML function
def parse_voc_xml(xp):
    """Parse Pascal VOC XML -> unified dict."""
    root = ET.fromstring(xp.read_text())

    # filenmae
    fn_node = root.find("filename")
    image_path = fn_node.text.strip() if fn_node is not None else None

    # size
    sz = root.find("size")
    w = h = None
    if sz is not None: 
        try:
            w = int(sz.find("width").text)
            h = int(sz.find("height").text)
        except Exception:
            pass

    # objects -> shapes
    unified_shapes = []
    for obj in root.findall("object"):
        name_node = obj.find("name")
        lab = name_node.text.strip() if name_node is not None else None
        bb = obj.find("bndbox")
        if lab and bb is not None:
            try:
                xmin = float(bb.find("xmin").text)
                ymin = float(bb.find("ymin").text)
                xmax = float(bb.find("xmax").text)
                ymax = float(bb.find("ymax").text)
            except Exception:
                continue

            # normalize to the same format as LabelMe: two-point rect
            rect = [[xmin, ymin], [xmax, ymax]]
            unified_shapes.append({"label": lab, "points": rect})
    
    return {
        "imagePath": image_path, # may need stem matching fallback later
        "imageWidth": w, 
        "imageHeight": h,
        "shapes": unified_shapes,
    }

In [46]:
# Collect JSONs and build a per-teammate index by (imagePath or image stem)

json_entries = []  # list of (teammate_name, json_path, data)
classes_set = set() # dynamically collect the unique class names (e.g. mattress, cans, bottles, couch, toys).


for tdir in teammates:
    # for each teammate directoy
    # look for an 'annotations' subfolder where LabelMe JSONs live.
    ann_dir = tdir / "annotations"
    if not ann_dir.exists():
        # if a teammate hasn't provide annotations yet, skip
        continue

    # iterate over all .json and .xml files in that annotations folder
    for ap in list(ann_dir.glob("*.json")) + list(ann_dir.glob("*.xml")):
        try:
            # read and parse the JSON file into a Python dict
            if ap.suffix.lower() == ".json":
                data = parse_labelme_json(ap)
            else: # ".xml"
                data = parse_voc_xml(ap)
        except Exception as e:
            # if the JSON is malformed or unreadable, don't crash the pipeline
            # print a help message and skip the file
            print(f"Skipping bad annotation files: {ap} ({e})")
            continue

        # store the trio (who/where/what) so later steps can:
        # - find the corresponding image in that teammate's 'rubbush/' folder
        # - convert shapes -> YOLO labels
        json_entries.append((tdir.name, ap, data))

        # collect class labels from the JSON
        # LabelMe stores shapes (boxes or polygons) under 'shapes'
        for sh in data.get("shapes", []):
            lab = sh.get("label")
            if lab:
                # add to a set to ensure each class appears only one overall.
                classes_set.add(lab)


# sort the unordered set so class indices are stsable and reproducible
CLASSES = sorted(classes_set)

# diagnostics
print(f"\nDiscovered {len(CLASSES)} classes:", CLASSES)
print(f"Total annotation files discovered: {len(json_entries)}")


Discovered 60 classes: ['aluminium_cans', 'blanket', 'bottle', 'box', 'bumper', 'can', 'car_bumper', 'cardboard', 'carton', 'chair', 'clothes', 'couch', 'cup', 'dirt', 'electrical goods', 'electrical_goods', 'electronic_goods', 'furniture', 'furniture_scraps', 'garbage', 'glass_bottle', 'guide_sign', 'guide_sign_bent', 'guide_sign_rust', 'human', 'jug', 'litter', 'mail', 'mattress', 'non-damaged-sign', 'paper_cup', 'plant_waste', 'plastic_bottle', 'regulatory_sign', 'regulatory_sign_graffiti', 'restriction_sign', 'restriction_sign_bent', 'restriction_sign_rust', 'rubbish bag', 'rubbish_bag', 'rug', 'scrap', 'speed_sign', 'speed_sign_bent', 'speed_sign_graffiti', 'stray_trolley', 'styrofoam', 'syringe', 'television', 'toilet_seat', 'torn_paper', 'toy', 'trash_can', 'trolley', 'warning_sign', 'warning_sign_bent', 'warning_sign_graffiti', 'warning_sign_rust', 'waste', 'wooden_pallet']
Total annotation files discovered: 658


In [47]:
# canonical classes + alias map and a normalizer
CANONICAL = [
    "mattress",
    "electrical_goods",
    "couch",
    "trolley",
    "toy",
    "clothes",
    "cardboard",
    "rubbish_bag",
    "furniture",
    "litter",
    "carton",
    "aluminium_cans",
    "blanket",
    "bottle",
    'bumper',
    'mail',
    'cup',
    'waste',
    'styrofoam',
    'toilet_seat',
    'wooden_pallet',
    'trash_can',
]

# handcrafted aliases (typos and common name mismatches)
ALIASES = {
    # spacing/hyphens/underscores variants
    "rubbish bag": "rubbish_bag",
    'electrical goods': "electrical_goods",
    'electronic_goods': "electrical_goods",
    
    # common mislabels / synonyms
    'paper_cup': "cup",
    'car_bumper': "bumper",
    'box': "cardboard",
    'can': "aluminium_cans",
    "stray_trolley": "trolley",
    "scrap": "litter",
    "jug": "carton",
    "garbage": "rubbish_bag",
    "furniture_scraps": "furniture",
    "chair": "furniture",
    "glass_bottle": "bottle",
    "plastic_bottle": "bottle",
    "torn_paper": "litter",
    "television": "electrical_goods",
    'torn_paper': 'waste',
    'rug': 'blanket',
    'dirt': 'waste',
    'plant_waste': 'waste',
    'syringe': 'waste',
}

# pre-compute a lowercase lookup for canonical names
CANON_LOWER = {c.lower(): c for c in CANONICAL}

def canonicalize(raw_label: str, fuzzy=True):
    """
    Convert raw teammate label -> canonical class name or None if unknown.
    Strat:
        1) normalize spaces/hyphens/underscores, lowercase
        2) apply ALIASES
        3) exact match to CANONICAL
    """

    if not raw_label:
        return None
    
    s = raw_label.strip().lower()
    s = re.sub(r"[\s\-]+", "_", s) # spaces/hyphens -> underscore
    s = re.sub(r"__+", "_", s) # collapse doubles

    # direct alias
    if s in ALIASES:
        s = ALIASES[s]

    # exact canonical match
    if s in CANON_LOWER:
        return CANON_LOWER[s]
    
    return None

In [48]:
for i, (tname, ap, data) in enumerate(json_entries):
    pts = data.get("shapes",[])
    print(pts[0].get('label'))
    break

furniture_scraps


In [49]:
from collections import Counter, defaultdict

normalized_counts = Counter()
unknown_examples = defaultdict(int)

total_shapes_before = 0
total_shapes_after = 0
NEW_CLASSES = set()
for i, (tname, ap, data) in enumerate(json_entries):
    new_shapes = []
    for sh in data.get("shapes", []):
        total_shapes_before += 1
        raw = sh.get("label")
        pts = sh.get("points")
        if not raw or not pts:
            continue
        
        canon = canonicalize(raw)
        print(f"Raw -> Canonical: '{raw}' -> '{canon}'")
        if canon is None:
            # keep a small log to fix upstream labels later
            key = f"{raw} -> ? (in {ap.name})"
            unknown_examples[key] += 1
            continue

        normalized_counts[canon] += 1
        new_shapes.append({"label": canon, "points": sh["points"]})
        NEW_CLASSES.add(canon)
    data["shapes"] = new_shapes
    json_entries[i] = (tname, ap, data)

    total_shapes_after += len(new_shapes)

print("normalization complete")
print("shapes before:", total_shapes_before, "after:", total_shapes_after)
print("\nclass counts after normalization:")
for cls in CANONICAL:
    print(f"{cls:16s} {normalized_counts[cls]}")

if unknown_examples:
    print("\n unknown/mismatched labels (top 15):")
    for k, v in list(sorted(unknown_examples.items(), key=lambda x: -x[1])):
        print(f" {v:4d} {k}")

else:
    print("\n all good")

Raw -> Canonical: 'furniture_scraps' -> 'furniture'
Raw -> Canonical: 'electrical goods' -> 'electrical_goods'
Raw -> Canonical: 'chair' -> 'furniture'
Raw -> Canonical: 'couch' -> 'couch'
Raw -> Canonical: 'couch' -> 'couch'
Raw -> Canonical: 'litter' -> 'litter'
Raw -> Canonical: 'mattress' -> 'mattress'
Raw -> Canonical: 'couch' -> 'couch'
Raw -> Canonical: 'couch' -> 'couch'
Raw -> Canonical: 'furniture_scraps' -> 'furniture'
Raw -> Canonical: 'litter' -> 'litter'
Raw -> Canonical: 'furniture' -> 'furniture'
Raw -> Canonical: 'mattress' -> 'mattress'
Raw -> Canonical: 'chair' -> 'furniture'
Raw -> Canonical: 'cardboard' -> 'cardboard'
Raw -> Canonical: 'cardboard' -> 'cardboard'
Raw -> Canonical: 'couch' -> 'couch'
Raw -> Canonical: 'aluminium_cans' -> 'aluminium_cans'
Raw -> Canonical: 'couch' -> 'couch'
Raw -> Canonical: 'carton' -> 'carton'
Raw -> Canonical: 'furniture_scraps' -> 'furniture'
Raw -> Canonical: 'furniture_scraps' -> 'furniture'
Raw -> Canonical: 'electrical goods'

In [118]:
NEW_CLASSES = list(NEW_CLASSES)

In [ ]:
def class_counts(json_entries):
    counts = Counter()
    for tname, ap, data in json_entries:
        for sh in data.get("shapes", []):
            lab = sh.get("label")
            if lab:
                counts[lab] += 1
    return counts
count = class_counts(json_entries)
ordered_counts = [count[c] for c in NEW_CLASSES]
weights = [1.0/c for c in ordered_counts]
weights

[0.0196078431372549,
 0.2,
 0.038461538461538464,
 0.05555555555555555,
 0.024390243902439025,
 0.00625,
 0.02564102564102564,
 0.2,
 0.005780346820809248,
 0.04,
 0.07142857142857142,
 0.030303030303030304,
 0.008849557522123894,
 0.014492753623188406,
 0.3333333333333333,
 0.012987012987012988,
 0.0036101083032490976,
 0.09090909090909091,
 0.02702702702702703,
 1.0,
 0.09090909090909091,
 0.004784688995215311]

In [56]:
# merge annotated images (rubbish with JSON), then add balanced not_rubbish

labels_written = 0
images_copied = 0

# book-keeping:
merged_list = []        # all merged image paths
pos_image_ids = set()   # stems of positive (rubbish) images after unique-naming
neg_image_ids = set()   # stems of negative (not_rubbish) images after unique-naming

def find_rubbish_image(teammate_dir: Path, image_filename: str):
    """
    given a teammate's root folder and an annotation's image filename (imagePath),
    try to find the *actual* image in teammate/rubbish/.
    1) first attempt exact match (same name + extension)
    2) if not found, fall back to matching by stem (ignore extension), so 'foo.jpg' can match 'foo.PNG'
    """
    rub_dir = teammate_dir / "rubbish"
    
    # exact filename match
    if not rub_dir.exists(): return None
    cand = rub_dir / image_filename
    if cand.exists(): return cand

    # fallback: match by stem (any extension)
    stem = Path(image_filename).stem
    for p in rub_dir.iterdir():
        if is_image_file(p) and p.stem == stem:
            return p
    return None


# merge ONLY annotated rubbish images ----
for teammate_name, jp, data in json_entries:
    # prefer labelmel's recorded image name; if missing try same name as JSON but '.jpg'
    image_fn = data.get("imagePath") or jp.with_suffix(".jpg").name  # heuristic fallback
    teammate_dir = ROOT / teammate_name

    # find the matching rubbish image file for this annotation JSON
    img_path = find_rubbish_image(teammate_dir, image_fn)
    if img_path is None:
        # if we can't find the image, skip without error
        print(f"No matching rubbish image for JSON: {jp.name} (expected {image_fn})")
        continue

    # open to get size for normalization when writing to YOLO labels
    try:
        with Image.open(img_path) as im:
            w, h = im.size
    except Exception as e:
        # don't let a corrupt image through
        print(f"Cannot open image {img_path}: {e}")
        continue

    # a unique base name to avoid collisions between teammates
    ub = unique_name(teammate_name, img_path.name)  # unique base
    out_img = (MERGED / "images" / f"{ub}{img_path.suffix.lower()}")

    # copy the image into the merged dataset if we haven't yet
    if not out_img.exists():
        shutil.copy2(img_path, out_img)
        images_copied += 1
        merged_list.append(out_img)
        pos_image_ids.add(out_img.stem) # remember this positive's stem

    # write YOLO label
    out_lbl = (MERGED / "labels" / f"{ub}.txt")
    with open(out_lbl, "a") as f:
        # each shape in LabelMe is a labled region
        for sh in data.get("shapes", []):
            lab = sh.get("label")
            pts = sh.get("points", [])
            # skip if label is empty, no points, or label not in our discovered class list
            # if not lab or not pts or lab not in CLASSES:
            if not lab or not pts or lab not in NEW_CLASSES:
                continue

            # map class name -> numeric class id via CLASSES ordering
            # cls_id = CLASSES.index(lab)
            cls_id = NEW_CLASSES.index(lab)

            # convert polygon/rect points into a tight bounding box
            xmin, ymin, xmax, ymax = bbox_from_points(pts)

            # convert that box into YOLO normalized "class xc, yc, w, h" line
            line = to_yolo_line(cls_id, (xmin, ymin, xmax, ymax), w, h)
            if line:
                f.write(line + "\n")
                labels_written += 1


# diagnose
num_positives = len(pos_image_ids)
print(f"Positive (annotated rubbish) images merged: {num_positives}")
print(f"YOLO label lines written: {labels_written}")

# ddd balanced not_rubbish (empty labels) ----
if ADD_NOT_RUBBISH_BALANCED:
    # Gather all available not_rubbish images across teammates
    pool = []
    for tdir in [d for d in ROOT.iterdir() if d.is_dir()]:
        nr_dir = tdir / "not_rubbish"
        if not nr_dir.exists(): 
            continue
        for p in nr_dir.iterdir():
            if is_image_file(p):
                pool.append((tdir.name, p))
    print(f"Found {len(pool)} candidate not_rubbish images across teammates.")

    need = int(num_positives * MAX_NOT_RUBBISH_PER_POSITIVE)
    if need <= 0:
        print("No negatives requested (need <= 0).")
    else:
        # Sample without replacement (cap at available)
        random.shuffle(pool)
        take = min(need, len(pool))
        sampled = pool[:take]

        added_nr = 0
        for teammate_name, p in sampled:
            # build a unique name for the negative image too
            ub = unique_name(teammate_name, p.name)
            out_img = (MERGED / "images" / f"{ub}{p.suffix.lower()}")

            # if by change a positive already used this unique ame, skip
            if out_img.exists():   # avoid collision with a positive of same ub
                continue
            shutil.copy2(p, out_img)
            open(MERGED / "labels" / f"{ub}.txt", "w").close()  # empty label
            neg_image_ids.add(out_img.stem)
            merged_list.append(out_img)
            added_nr += 1

        print(f"Added not_rubbish (negatives): {added_nr} (requested {need})")


No matching rubbish image for JSON: damaged-sign3-001.xml (expected damaged-sign3-001.png)
No matching rubbish image for JSON: damaged-sign3-002.xml (expected damaged-sign3-002.png)
No matching rubbish image for JSON: damaged-sign3-003.xml (expected damaged-sign3-003.png)
No matching rubbish image for JSON: damaged-sign3-004.xml (expected damaged-sign3-004.png)
No matching rubbish image for JSON: damaged-sign3-005.xml (expected damaged-sign3-005.png)
No matching rubbish image for JSON: damaged-sign3-006.xml (expected damaged-sign3-006.png)
No matching rubbish image for JSON: damaged-sign3-007.xml (expected damaged-sign3-007.png)
No matching rubbish image for JSON: damaged-sign3-008.xml (expected damaged-sign3-008.png)
No matching rubbish image for JSON: damaged-sign3-009.xml (expected damaged-sign3-009.png)
No matching rubbish image for JSON: damaged-sign3-010.xml (expected damaged-sign3-010.png)
No matching rubbish image for JSON: damaged-sign3-011.xml (expected damaged-sign3-011.png)

In [ ]:
   # save classes and data.yaml (YOYO-ready)
   # classes.txt

(CLASS_TXT := MERGED / "classes.txt").write_text("\n".join(NEW_CLASSES))
print("wrote", CLASS_TXT)

# data.yaml for YOLO/Ultralytics
yaml_text = f"""# Auto-generated data.yaml
path: {MERGED.resolve()}
train: images/train
val: images/val
cls_weight: {weights}
names:
"""

for i, name in enumerate(NEW_CLASSES):
    yaml_text += f" {i}: {name}\n"

(DATA_YAML := MERGED / "data.yaml").write_text(yaml_text)
print("wrote", DATA_YAML)

wrote merged_dataset\classes.txt
wrote merged_dataset\data.yaml


: 

In [59]:
# split merged images into train/val and move pairs

all_imgs = sorted((MERGED / "images").glob("*.*"))
random.shuffle(all_imgs)

split_idx = int(TRAIN_RATIO * len(all_imgs))
train_imgs = all_imgs[:split_idx]
val_imgs = all_imgs[split_idx:]

def move_pair(img_path: Path, phase: str):
    dst_img = MERGED / "images" / phase / img_path.name
    lbl_src = MERGED / "labels" / (img_path.stem + ".txt")
    dst_lbl = MERGED / "labels" / phase / (img_path.stem + ".txt")
    shutil.move(str(img_path), str(dst_img))
    if lbl_src.exists():
        shutil.move(str(lbl_src), str(dst_lbl))


for p in train_imgs:
    move_pair(p, "train")
for p in val_imgs:
    move_pair(p, "val")

print(f"train images: {len(train_imgs)}")
print(f"val images: {len(val_imgs)}")

train images: 500
val images: 125


## **Data Augmentation & Data Balancing goes here**


## **Model Development**

In [32]:
from ultralytics import YOLO

DATA_YAML = MERGED / "data.yaml"

assert MERGED.exists(), "merged_dataset not found"
assert DATA_YAML.exists(), "data.yaml missing"

print("using dataset:", MERGED.resolve())
print(DATA_YAML.read_text())

using dataset: D:\Studying\Swinburne-Studying\2025-Semester-2\COS40007-AI_for_Engineering\Smart-City-Civil-and-Construction-Engineering\merged_dataset
# Auto-generated data.yaml
path: D:\Studying\Swinburne-Studying\2025-Semester-2\COS40007-AI_for_Engineering\Smart-City-Civil-and-Construction-Engineering\merged_dataset
train: images/train
val: images/val

names:
 0: aluminium_cans
 1: blanket
 2: bottle
 3: box
 4: bumper
 5: can
 6: car_bumper
 7: cardboard
 8: carton
 9: chair
 10: clothes
 11: couch
 12: cup
 13: dirt
 14: electrical goods
 15: electrical_goods
 16: electronic_goods
 17: furniture
 18: furniture_scraps
 19: garbage
 20: glass_bottle
 21: guide_sign
 22: guide_sign_bent
 23: guide_sign_rust
 24: human
 25: jug
 26: litter
 27: mail
 28: mattress
 29: non-damaged-sign
 30: paper_cup
 31: plant_waste
 32: plastic_bottle
 33: regulatory_sign
 34: regulatory_sign_graffiti
 35: restriction_sign
 36: restriction_sign_bent
 37: restriction_sign_rust
 38: rubbish bag
 39: rub

In [12]:
# YOLOv8 model size selection
# start with 'n' (nano) for quick iteration; we can upgrade to 's' or 'm' for a bigger model size

base_weights = "yolov8s.pt" 
model = YOLO(base_weights)

# model summary
model

100%|██████████| 21.5M/21.5M [00:00<00:00, 26.3MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [13]:
import numpy

# training starts here
# important:
#   - epoch: 50
#   - imgsz: 640
#   - batch: typically 8-32
#   = device: "0" for gpu, or "cpu" to force CPU

results = model.train(
    data=str(DATA_YAML),
    epochs=90,
    imgsz=960,
    batch=8,
    workers=4,
    patience=8, # early stopping
    optimizer="AdamW",
    lr0=0.002, # initial LR
    # mild colot augmentation
    hsv_h=0.015, # hue
    hsv_s=0.7, # saturation
    hsv_v=0.4, # vibrance
    mosaic=0.1, # keep small (street scenes can get weird with high mosaic)
    weight_decay=0.0005,
    project="runs_theme2",
    name="yolov8s_rubbish",
    device="cpu"
)

New https://pypi.org/project/ultralytics/8.3.221 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.0  Python-3.10.18 torch-2.2.2+cpu CPU (13th Gen Intel Core(TM) i5-13400F)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=merged_dataset\data.yaml, epochs=90, time=None, patience=8, batch=8, imgsz=960, save=True, save_period=-1, cache=False, device=cpu, workers=4, project=runs_theme2, name=yolov8s_rubbish, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show

100%|██████████| 755k/755k [00:00<00:00, 64.4MB/s]

Overriding model.yaml nc=80 with nc=24

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

Model summary: 249 layers, 9,848,248 parameters, 9,848,232 gradients, 23.6 GFLOPs

Transferred 313/391 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\krazy\OneDrive - Swinburne University\Studying\Swinburne Studying\2025-Semester_2\COS40007-AI_for_Engineering\COS40007-Smart_City-Civil_and_Construction_Engineering\merged_dataset\labels\train... 99 images, 19 backgrounds, 0 corrupt: 100%|██████████| 99/99 [00:00<00:00, 276.34it/s]

...

      12/90         0G      1.883      3.393      2.094         11        960: 100%|██████████| 13/13 [01:57<00:00,  9.00s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:10<00:00,  5.18s/it]

                   all         25         56      0.446      0.019     0.0167    0.00867
EarlyStopping: Training stopped early as no improvement observed in last 8 epochs. Best results observed at epoch 4, best model saved as best.pt.
To update EarlyStopping(patience=8) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



12 epochs completed in 0.422 hours.
Optimizer stripped from runs_theme2\yolov8s_rubbish\weights\last.pt, 20.0MB
Optimizer stripped from runs_theme2\yolov8s_rubbish\weights\best.pt, 20.0MB

Validating runs_theme2\yolov8s_rubbish\weights\best.pt...
Ultralytics 8.3.0  Python-3.10.18 torch-2.2.2+cpu CPU (13th Gen Intel Core(TM) i5-13400F)
Model summary (fused): 186 layers, 9,836,952 parameters, 0 gradients, 23.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:08<00:00,  4.49s/it]


                   all         25         56     0.0756     0.0857     0.0849     0.0305
        aluminium_cans          2          3          0          0          0          0
             cardboard          3          9          0          0          0          0
                carton          2          4          0          0          0          0
             furniture         12         15      0.029      0.267     0.0935     0.0631
                litter          6         10          0          0          0          0
              mattress          3          3        0.5      0.333      0.501       0.15
           rubbish_bag          5         12          0          0          0          0
Speed: 4.9ms preprocess, 305.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs_theme2\yolov8s_rubbish


In [14]:
# validate on the Val split and generate plots
val_results = model.val(
    data=str(DATA_YAML),
    imgsz=960,
    split="val",
    project="runs_theme1",
    name="yolov8s_rubbish_val",
    device="cpu"
)

print("validation metrics:")
print(val_results)

print("artifacts saved to:", val_results.save_dir)

Ultralytics 8.3.0  Python-3.10.18 torch-2.2.2+cpu CPU (13th Gen Intel Core(TM) i5-13400F)
Model summary (fused): 186 layers, 9,836,952 parameters, 0 gradients, 23.4 GFLOPs


val: Scanning C:\Users\krazy\OneDrive - Swinburne University\Studying\Swinburne Studying\2025-Semester_2\COS40007-AI_for_Engineering\COS40007-Smart_City-Civil_and_Construction_Engineering\merged_dataset\labels\val.cache... 25 images, 2 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.66s/it]


                   all         25         56     0.0513     0.0857     0.0765     0.0275
        aluminium_cans          2          3          0          0          0          0
             cardboard          3          9          0          0          0          0
                carton          2          4          0          0          0          0
             furniture         12         15      0.026      0.267     0.0899     0.0586
                litter          6         10          0          0          0          0
              mattress          3          3      0.333      0.333      0.446      0.134
           rubbish_bag          5         12          0          0          0          0
Speed: 2.1ms preprocess, 215.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs_theme1\yolov8s_rubbish_val
validation metrics:
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  3,  4,  8, 13, 14, 17])
box: ultralytics.util

In [15]:
val_dir = MERGED / "images" / "val"
sample_imgs = list(val_dir.glob("*.*"))

pred = model.predict(
    source=[str(p) for p in sample_imgs],
    imgsz=960,
    conf=0.25,
    iou=0.5,
    save=True,
    project="runs_theme1",
    name="yolov8s_rubbish_preview",
    device="cpu"
)

print("preview images saved to:", pred[0].save_dir if pred else "no predictions")


0: 960x960 (no detections), 305.4ms
1: 960x960 (no detections), 305.4ms
2: 960x960 (no detections), 305.4ms
3: 960x960 (no detections), 305.4ms
4: 960x960 (no detections), 305.4ms
5: 960x960 (no detections), 305.4ms
6: 960x960 (no detections), 305.4ms
7: 960x960 (no detections), 305.4ms
8: 960x960 (no detections), 305.4ms
9: 960x960 (no detections), 305.4ms
10: 960x960 (no detections), 305.4ms
11: 960x960 (no detections), 305.4ms
12: 960x960 (no detections), 305.4ms
13: 960x960 (no detections), 305.4ms
14: 960x960 (no detections), 305.4ms
15: 960x960 (no detections), 305.4ms
16: 960x960 (no detections), 305.4ms
17: 960x960 (no detections), 305.4ms
18: 960x960 (no detections), 305.4ms
19: 960x960 (no detections), 305.4ms
20: 960x960 (no detections), 305.4ms
21: 960x960 (no detections), 305.4ms
22: 960x960 (no detections), 305.4ms
23: 960x960 (no detections), 305.4ms
24: 960x960 (no detections), 305.4ms
Speed: 6.1ms preprocess, 305.4ms inference, 0.4ms postprocess per image at shape (1,